In [1]:
#Import Dependencies
import pandas as pd
import pymongo
import numpy as np
import requests
import sys
import os.path
import datetime as dt
import pprint

import quandl
from config import quandl_api_key

from bs4 import BeautifulSoup as bs
from pymongo import MongoClient





In [2]:
try:
    us_yield_curve = quandl.get("USTREASURY/YIELD", authtoken=quandl_api_key)
    sp_500_div_yield_month = quandl.get("MULTPL/SP500_DIV_YIELD_MONTH", authtoken=quandl_api_key)
    sp_500_1m_futures = quandl.get("CHRIS/CME_SP1", authtoken=quandl_api_key)
    us_yield_curve.to_csv("quandl_data/treas_yld_curve.csv")
    sp_500_div_yield_month.to_csv("quandl_data/sp500_div_yld.csv")
    sp_500_1m_futures.to_csv("quandl_data/sp500_1m.csv")

except Exception as e:
    print(e)

In [3]:
us_yield_curve = us_yield_curve.sort_index(axis=0,ascending=False)
us_yield_curve_df = us_yield_curve.reset_index()
#us_yield_curve_df = us_yield_curve_df.sort_values(by="Date", ascending=False)
us_yield_curve_df

,Date,1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
0,2020-06-08,0.15,0.15,0.17,0.19,0.19,0.22,0.29,0.45,0.69,0.88,1.43,1.65
1,2020-06-05,0.13,0.14,0.15,0.18,0.18,0.22,0.29,0.47,0.71,0.91,1.46,1.68
2,2020-06-04,0.13,0.15,0.15,0.18,0.17,0.19,0.26,0.40,0.63,0.82,1.38,1.61
3,2020-06-03,0.12,0.14,0.16,0.19,0.17,0.19,0.26,0.38,0.59,0.77,1.32,1.56
4,2020-06-02,0.12,0.13,0.15,0.18,0.17,0.17,0.20,0.32,0.52,0.68,1.24,1.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7609,1990-01-08,NaN,NaN,7.79,7.88,7.81,7.90,7.95,7.92,8.05,8.02,NaN,8.09
7610,1990-01-05,NaN,NaN,7.79,7.85,7.79,7.90,7.94,7.92,8.03,7.99,NaN,8.06
7611,1990-01-04,NaN,NaN,7.84,7.90,7.82,7.92,7.93,7.91,8.02,7.98,NaN,8.04
7612,1990-01-03,NaN,NaN,7.89,7.94,7.85,7.94,7.96,7.92,8.04,7.99,NaN,8.04


In [4]:
sp_500_div_yield_month = sp_500_div_yield_month.sort_index(axis=0,ascending=False)
sp_500_div_yield_month_df = sp_500_div_yield_month.reset_index()
sp_500_div_yield_month_df = sp_500_div_yield_month_df.drop([0])
sp_500_div_yield_month_df

,Date,Value
1,2020-06-01,1.95
2,2020-05-31,2.03
3,2020-05-01,2.10
4,2020-04-30,2.16
5,2020-04-01,2.41
...,...,...
1794,1871-05-31,5.35
1795,1871-04-30,5.49
1796,1871-03-31,5.64
1797,1871-02-28,5.78


In [5]:
sp_500_1m_futures = sp_500_1m_futures.sort_index(axis=0,ascending=False)
sp_500_1m_futures_df = sp_500_1m_futures.reset_index()
sp_500_1m_futures_df

,Date,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest
0,2020-06-05,3114.40,3187.90,3106.80,NaN,76.3,3186.80,7645.0,108833.0
1,2020-06-04,3113.90,3126.20,3094.80,NaN,7.2,3110.50,1696.0,108199.0
2,2020-06-03,3076.80,3118.50,3074.90,NaN,40.8,3117.70,2421.0,107654.0
3,2020-06-02,3051.30,3078.40,3035.50,NaN,22.9,3076.90,2064.0,106077.0
4,2020-06-01,3033.50,3054.70,3008.70,NaN,12.0,3054.00,847.0,105797.0
...,...,...,...,...,...,...,...,...,...
9594,1982-04-27,120.20,120.35,118.40,118.85,NaN,118.85,3854.0,1142.0
9595,1982-04-26,119.30,120.60,118.65,120.55,NaN,120.55,2871.0,1084.0
9596,1982-04-23,118.40,119.75,118.25,119.65,NaN,119.65,2933.0,946.0
9597,1982-04-22,117.00,118.40,117.00,117.90,NaN,117.90,2827.0,719.0


In [6]:
#Merge the Data using Outer method

sp_500_futures_vs_div_df = pd.merge(sp_500_1m_futures_df, sp_500_div_yield_month_df, on="Date", how="outer")
sp_500_futures_vs_div_df = pd.merge(sp_500_futures_vs_div_df, us_yield_curve_df, on="Date", how="outer")


sp_500_futures_vs_div_df = pd.DataFrame({
    "Date": sp_500_futures_vs_div_df["Date"],
    "S&P 500 Close Price": sp_500_futures_vs_div_df["Settle"],
    "S&P 500 Dividend Yield": sp_500_futures_vs_div_df["Value"],
    "2-Yr Treasury Yield": sp_500_futures_vs_div_df["2 YR"],
    "10-Yr Treasury Yield": sp_500_futures_vs_div_df["10 YR"],
    "10Yr - 2Yr Treasury Yield": sp_500_futures_vs_div_df["10 YR"] - sp_500_futures_vs_div_df["2 YR"]
})

print(sp_500_futures_vs_div_df.shape)
sp_500_futures_vs_div_df

(11098, 6)


,Date,S&P 500 Close Price,S&P 500 Dividend Yield,2-Yr Treasury Yield,10-Yr Treasury Yield,10Yr - 2Yr Treasury Yield
0,2020-06-05,3186.8,NaN,0.22,0.91,0.69
1,2020-06-04,3110.5,NaN,0.19,0.82,0.63
2,2020-06-03,3117.7,NaN,0.19,0.77,0.58
3,2020-06-02,3076.9,NaN,0.17,0.68,0.51
4,2020-06-01,3054.0,1.95,0.14,0.66,0.52
...,...,...,...,...,...,...
11093,2001-09-14,NaN,NaN,2.87,4.57,1.70
11094,2001-09-13,NaN,NaN,2.99,4.64,1.65
11095,1999-04-02,NaN,NaN,4.96,5.20,0.24
11096,1996-04-05,NaN,NaN,6.09,6.57,0.48


In [7]:
#Sort the Data by Index

sp_500_futures_vs_div_df_sorted = sp_500_futures_vs_div_df.sort_index(axis=0,ascending=False)
sp_500_futures_vs_div_df_sorted

,Date,S&P 500 Close Price,S&P 500 Dividend Yield,2-Yr Treasury Yield,10-Yr Treasury Yield,10Yr - 2Yr Treasury Yield
11097,1993-09-01,NaN,NaN,3.88,5.46,1.58
11096,1996-04-05,NaN,NaN,6.09,6.57,0.48
11095,1999-04-02,NaN,NaN,4.96,5.20,0.24
11094,2001-09-13,NaN,NaN,2.99,4.64,1.65
11093,2001-09-14,NaN,NaN,2.87,4.57,1.70
...,...,...,...,...,...,...
4,2020-06-01,3054.0,1.95,0.14,0.66,0.52
3,2020-06-02,3076.9,NaN,0.17,0.68,0.51
2,2020-06-03,3117.7,NaN,0.19,0.77,0.58
1,2020-06-04,3110.5,NaN,0.19,0.82,0.63


In [8]:
sp_500_futures_vs_div_df_sorted.dtypes

Date                         datetime64[ns]
S&P 500 Close Price                 float64
S&P 500 Dividend Yield              float64
2-Yr Treasury Yield                 float64
10-Yr Treasury Yield                float64
10Yr - 2Yr Treasury Yield           float64
dtype: object

In [9]:
#Change Date type to "datetime" for further sorting by date range

sp_500_futures_vs_div_df_sorted["Date"] = pd.to_datetime(sp_500_futures_vs_div_df_sorted["Date"])
sp_500_futures_vs_div_df_sorted.dtypes

Date                         datetime64[ns]
S&P 500 Close Price                 float64
S&P 500 Dividend Yield              float64
2-Yr Treasury Yield                 float64
10-Yr Treasury Yield                float64
10Yr - 2Yr Treasury Yield           float64
dtype: object

In [10]:
#Cleaning data to start at earliest common start date for calculations

sp_500_futures_vs_div_df_sorted = sp_500_futures_vs_div_df_sorted[sp_500_futures_vs_div_df_sorted["S&P 500 Close Price"].notna()]
column_value = sp_500_futures_vs_div_df_sorted[sp_500_futures_vs_div_df_sorted["Date"] < "1990-01-31"].index
sp_500_futures_vs_div_df_sorted.drop(column_value, inplace=True)
sp_500_futures_vs_div_df_sorted

C:\Users\garyf\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Date,S&P 500 Close Price,S&P 500 Dividend Yield,2-Yr Treasury Yield,10-Yr Treasury Yield,10Yr - 2Yr Treasury Yield
7630,1990-01-31,330.50,3.28,8.28,8.43,0.15
7629,1990-02-01,330.10,NaN,8.28,8.42,0.14
7628,1990-02-02,332.50,NaN,8.37,8.50,0.13
7627,1990-02-05,333.40,NaN,8.37,8.53,0.16
7626,1990-02-06,330.25,NaN,8.42,8.57,0.15
...,...,...,...,...,...,...
4,2020-06-01,3054.00,1.95,0.14,0.66,0.52
3,2020-06-02,3076.90,NaN,0.17,0.68,0.51
2,2020-06-03,3117.70,NaN,0.19,0.77,0.58
1,2020-06-04,3110.50,NaN,0.19,0.82,0.63


In [11]:
#Calculate Daily Dividend Yield from the monthly value

total_yield_list = []
dollar_yield_list = []

for index, row in sp_500_futures_vs_div_df_sorted.iterrows():
    price_row = row["S&P 500 Close Price"]
    yield_row = row["S&P 500 Dividend Yield"]
    if yield_row > 0:
        total_yield_list.append(round(yield_row, 2))
        current_yield = yield_row
        dollar_yield = (current_yield / 100) * price_row
        dollar_yield_list.append(round(dollar_yield, 2))
    else:
        percent_yield = dollar_yield / price_row *100
        total_yield_list.append(round(percent_yield, 2))
    
print(len(total_yield_list))   

7631


In [12]:
#Calculate daily percent returns

sp_500_futures_vs_div_df_sorted["S&P 500 Dividend Yield"] = total_yield_list

sp500_percent_return = []
sp500_div_yield_return = []
treas_10yr_yield_return = []
treas_2yr_yield_return = []
treas_10yr2yr_yield_return = []

previous_price = 325.20
previous_div_yield = 3.28
previous_10yr_treas_yield = 8.51
previous_2yr_treas_yield = 8.30
previous_10yr2yr_treas_yield = 0.21

for index, row in sp_500_futures_vs_div_df_sorted.iterrows():
    price_row = row["S&P 500 Close Price"]
    div_yield_row = row["S&P 500 Dividend Yield"]
    treas_10yr_yield_row = row["10-Yr Treasury Yield"]
    treas_2yr_yield_row = row["2-Yr Treasury Yield"]
    treas_10yr2yr_yield_row = row["10Yr - 2Yr Treasury Yield"]
    
    
    daily_return_sp500 = (price_row - previous_price) / previous_price * 100
    sp500_percent_return.append(round(daily_return_sp500, 2))
    previous_price = price_row
    
    daily_return_div_yield = (div_yield_row - previous_div_yield) / previous_div_yield * 100
    sp500_div_yield_return.append(round(daily_return_div_yield, 2))
    previous_div_yield = div_yield_row
    
    daily_return_treas_10yr_yield = (treas_10yr_yield_row - previous_10yr_treas_yield) / previous_10yr_treas_yield * 100
    treas_10yr_yield_return.append(round(daily_return_treas_10yr_yield, 2))
    previous_10yr_treas_yield = treas_10yr_yield_row
    
    daily_return_treas_2yr_yield = (treas_2yr_yield_row - previous_2yr_treas_yield) / previous_2yr_treas_yield * 100
    treas_2yr_yield_return.append(round(daily_return_treas_2yr_yield, 2))
    previous_2yr_treas_yield = treas_2yr_yield_row
    
    daily_return_treas_10yr2yr_yield = (treas_10yr2yr_yield_row - previous_10yr2yr_treas_yield) / previous_10yr2yr_treas_yield * 100
    treas_10yr2yr_yield_return.append(round(daily_return_treas_10yr2yr_yield, 2))
    if treas_10yr2yr_yield_row == 0:
        previous_10yr2yr_treas_yield = 0.00000000000000000001
    else:
        previous_10yr2yr_treas_yield = treas_10yr2yr_yield_row


print(len(sp500_percent_return))
print(len(sp500_div_yield_return))
print(len(treas_10yr_yield_return))
print(len(treas_2yr_yield_return))
print(len(treas_10yr2yr_yield_return))

C:\Users\garyf\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


7631
7631
7631
7631
7631


In [ ]:
#Insert daily percent returns into dataframe

sp_500_futures_vs_div_df_sorted["S&P 500 Dividend Yield"] = total_yield_list

sp_500_futures_vs_div_df_sorted.insert(2, "S&P 500 % Return", sp500_percent_return)
sp_500_futures_vs_div_df_sorted.insert(4, "Div Yield % Return", sp500_div_yield_return)
sp_500_futures_vs_div_df_sorted.insert(6, "2-Yr Treasury Yield % Return", treas_2yr_yield_return)
sp_500_futures_vs_div_df_sorted.insert(8, "10-Yr Treasury Yield % Return", treas_10yr_yield_return)

sp_500_futures_vs_div_df_sorted["10Yr - 2Yr Treasury % Return"] = treas_10yr2yr_yield_return

sp_500_futures_vs_div_df_sorted.head(5)

In [ ]:
#Check for NaN values, and shape of dataframe

print(sp_500_futures_vs_div_df_sorted.isnull().sum())
print(sp_500_futures_vs_div_df_sorted.shape)


In [ ]:
#Drop NaN rows from dataframe

sp_500_futures_vs_div_df_sorted.dropna(inplace=True)
print(sp_500_futures_vs_div_df_sorted.isnull().sum())
print(sp_500_futures_vs_div_df_sorted.shape)


In [ ]:
#Sort Dataframe by date range

analyze_dates_df = sp_500_futures_vs_div_df_sorted.loc[(sp_500_futures_vs_div_df_sorted["Date"] >= "1990-01-31") 
                                               & (sp_500_futures_vs_div_df_sorted["Date"] <= "2020-03-20")]
analyze_dates_df.head(5)

In [ ]:
#Calculate total returns for date range

sp500_tot_ret = []
sp500_divyld_tot_ret = []
treas_10yr_yld_tot_ret = []
treas_2yr_yld_tot_ret = []
treas_10yr2yr_yld_tot_ret = []
divyld_less_10yr_treas_tot_ret = []
divyld_less_2yr_treas_tot_ret = []
divyld_less_10yr2yr_treas_tot_ret = []


start_price = analyze_dates_df["S&P 500 Close Price"].iloc[0]
start_div_yield = analyze_dates_df["S&P 500 Dividend Yield"].iloc[0]
start_10yr_treas_yield = analyze_dates_df["10-Yr Treasury Yield"].iloc[0]
start_2yr_treas_yield = analyze_dates_df["2-Yr Treasury Yield"].iloc[0]
start_10yr2yr_treas_yield = analyze_dates_df["10Yr - 2Yr Treasury Yield"].iloc[0]

for index, row in analyze_dates_df.iterrows():
    price_row = row["S&P 500 Close Price"]
    div_yield_row = row["S&P 500 Dividend Yield"]
    treas_10yr_yield_row = row["10-Yr Treasury Yield"]
    treas_2yr_yield_row = row["2-Yr Treasury Yield"]
    treas_10yr2yr_yield_row = row["10Yr - 2Yr Treasury Yield"]
    
    
    tot_ret_sp500 = (price_row - start_price) / start_price * 100
    sp500_tot_ret.append(round(tot_ret_sp500, 2))
    
    tot_ret_div_yld = (div_yield_row - start_div_yield) / start_div_yield * 100
    sp500_divyld_tot_ret.append(round(tot_ret_div_yld, 2))
    
    tot_ret_10yr_treas_yield = (treas_10yr_yield_row - start_10yr_treas_yield) / start_10yr_treas_yield * 100
    treas_10yr_yld_tot_ret.append(round(tot_ret_10yr_treas_yield, 2))
    
    tot_ret_2yr_treas_yield = (treas_2yr_yield_row - start_2yr_treas_yield) / start_2yr_treas_yield * 100
    treas_2yr_yld_tot_ret.append(round(tot_ret_2yr_treas_yield, 2))
    
    tot_ret_10yr2yr_treas_yield = (treas_10yr2yr_yield_row - start_10yr2yr_treas_yield) / start_10yr2yr_treas_yield * 100
    treas_10yr2yr_yld_tot_ret.append(round(tot_ret_10yr2yr_treas_yield, 2))
    
    divyld_less_10yr_treas = tot_ret_div_yld - tot_ret_10yr_treas_yield
    divyld_less_10yr_treas_tot_ret.append(round(divyld_less_10yr_treas, 2))
    
    divyld_less_2yr_treas = tot_ret_div_yld - tot_ret_2yr_treas_yield
    divyld_less_2yr_treas_tot_ret.append(round(divyld_less_2yr_treas, 2))
    
    divyld_less_10yr2yr_treas = tot_ret_div_yld - tot_ret_10yr2yr_treas_yield
    divyld_less_10yr2yr_treas_tot_ret.append(round(divyld_less_10yr2yr_treas, 2))
    

print(start_price)
print(start_div_yield)
print(start_10yr_treas_yield)
print(start_2yr_treas_yield)
print(start_10yr2yr_treas_yield)
analyze_dates_df.head(1)

In [ ]:
#Create Master Dataframe for Charts

analyze_metrics_df = pd.DataFrame({
    "Date": analyze_dates_df["Date"],
    "S&P Price": analyze_dates_df["S&P 500 Close Price"],
    "S&P Div Yld": analyze_dates_df["S&P 500 Dividend Yield"],
    "2Yr Treas Yld": analyze_dates_df["2-Yr Treasury Yield"],
    "10Yr Treas Yld": analyze_dates_df["10-Yr Treasury Yield"],
    #"10Yr2Yr Treas Yld": analyze_dates_df["10-Yr Treasury Yield"] - analyze_dates_df["2-Yr Treasury Yield"],
    #"Div Yld - 2Yr Treas Yld": analyze_dates_df["S&P 500 Dividend Yield"] - analyze_dates_df["2-Yr Treasury Yield"],
    #"Div Yld - 10Yr Treas Yld": analyze_dates_df["S&P 500 Dividend Yield"] - analyze_dates_df["10-Yr Treasury Yield"],
    #"Div Yld - 10Yr2Yr Treas Yld": analyze_dates_df["S&P 500 Dividend Yield"] - analyze_dates_df["10Yr - 2Yr Treasury Yield"],
    "Div Yld / 10Yr2Yr Treas Yld": analyze_dates_df["S&P 500 Dividend Yield"] / analyze_dates_df["10Yr - 2Yr Treasury Yield"],
    "S&P Tot Ret": sp500_tot_ret,
    #"S&P Div Yld Tot Ret": sp500_divyld_tot_ret,
    "2Yr Treas Yld Tot Ret": treas_2yr_yld_tot_ret,
    "10Yr Treas Yld Tot Ret": treas_10yr_yld_tot_ret,
    #"10Yr2Yr Treas Yld Tot Ret": treas_10yr2yr_yld_tot_ret,
    #"Div Yld Tot Ret - 2Yr Treas Yld Tot Ret": divyld_less_2yr_treas_tot_ret,
    #"Div Yld Tot Ret - 10Yr Treas Yld Tot Ret": divyld_less_10yr_treas_tot_ret,
    #"Div Yld Tot Ret - 10Yr2Yr Treas Yld Tot Ret": divyld_less_10yr2yr_treas_tot_ret,
    "S&P Daily Ret": analyze_dates_df["S&P 500 % Return"],
    "S&P Div Yld Daily Ret": analyze_dates_df["Div Yield % Return"],
    #"2Yr Treas Yld Daily Ret": analyze_dates_df["2-Yr Treasury Yield % Return"],
    "10Yr Treas Yld Daily Ret": analyze_dates_df["10-Yr Treasury Yield % Return"],
    "10Yr2Yr Treas Yld Daily Ret": analyze_dates_df["10Yr - 2Yr Treasury % Return"],    
    #"Div Yld Daily Ret - 2Yr Treas Yld Daily Ret": analyze_dates_df["Div Yield % Return"] - analyze_dates_df["2-Yr Treasury Yield % Return"],
    #"Div Yld Daily Ret - 10Yr Treas Yld Daily Ret": analyze_dates_df["Div Yield % Return"] - analyze_dates_df["10-Yr Treasury Yield % Return"],
    #"Div Yld Daily Ret - 10Yr2YrTreas Yld Daily Ret": analyze_dates_df["Div Yield % Return"] - analyze_dates_df["10Yr - 2Yr Treasury % Return"]
})

analyze_metrics_df = analyze_metrics_df.replace([np.inf, -np.inf], np.nan)

print(analyze_metrics_df.isnull().sum())
analyze_metrics_df.dropna(inplace=True)
print(analyze_metrics_df.isnull().sum())
print(analyze_metrics_df.shape)
analyze_metrics_df.head(2)
#print(analyze_metrics_df.shape)